In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split, cross_val_predict, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, make_scorer, roc_auc_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
import random
from sklearn.neighbors import LocalOutlierFactor
from sklearn.feature_selection import RFECV, SelectKBest, SequentialFeatureSelector, f_classif, mutual_info_classif

from sklearn.impute import SimpleImputer, KNNImputer

from Functions_Classes import *

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE, RandomOverSampler

from feature_engine.encoding import CountFrequencyEncoder

from xgboost import XGBClassifier

import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
np.random.seed(0)

In [2]:
df = pd.read_excel("C:/Users/Cagan Deliktas/Desktop/ProjectDataMining2/DM2_DataCraft/data/training_data.xls")
X_test_compete = pd.read_excel("C:/Users/Cagan Deliktas/Desktop/ProjectDataMining2/DM2_DataCraft/data/test_data_no_target.xls")

df = df.loc[:, df.columns != 'Perform']
df = df.loc[:, df.columns != 'Group']


df_x = df.loc[:, df.columns != 'Class']
df_y = df['Class']

X_train = df_x.copy()
y_train =  df_y.copy()

#X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2,shuffle=True, stratify=df_y, random_state=0)

In [3]:
numeric_columns = X_train.loc[:, ~X_train.columns.isin(['Group'])].columns.to_list()
X_train[numeric_columns] = X_train[numeric_columns].replace(
    {
        'NA': np.nan, 
        '': np.nan,
        ' ': np.nan
    }
).astype(float)

In [4]:
X_train.head()

,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,I14,I15,I16,I17,I18,I19,I20,I21,I22,I23,I24,I25,I26,I27,I28,I29,I30,I31,I32,I33,I34,I35,I36,I37,I38,I39,I40,I41,I42,I43,I44,I45,I46,I47,I48,I49,I50,I51,I52,I53,I54,I55,I56,I57,I58,dI1,dI2,dI3,dI4,dI5,dI6,dI7,dI8,dI9,dI10,dI11,dI12,dI13,dI14,dI15,dI16,dI17,dI18,dI19,dI20,dI21,dI22,dI23,dI24,dI25,dI26,dI27,dI28,dI29,dI30,dI31,dI32,dI33,dI34,dI35,dI36,dI37,dI38,dI39,dI40,dI41,dI42,dI43,dI44,dI45,dI46,dI47,dI48,dI49,dI50,dI51,dI52,dI53,dI54,dI55,dI56,dI57,dI58
0,0.136495,-0.028429,-0.037772,-0.232459,-0.016222,-0.187506,-0.322545,-0.043743,0.125389,-0.014757,-0.033105,0.303035,-0.093811,-0.598917,-0.271292,-0.256749,-0.100146,-0.045525,-0.078422,-0.060129,-0.069528,-0.052432,-0.114432,-0.104989,0.342845,-0.159417,0.006772,-0.303193,-0.163287,-0.080599,-0.828880,-1.064215,-0.547067,-0.540497,-0.676045,-0.305007,-0.507724,-0.191437,-0.087362,-0.856151,0.802525,0.733080,0.006512,0.533290,0.195197,0.058094,-0.228889,-0.150821,-0.104986,-0.026743,0.188312,-0.250701,-0.101190,-0.357521,-0.527956,0.611385,-0.092714,-0.055733,-0.065709,-0.002144,-0.004367,-0.079805,0.178280,0.078155,0.072802,0.002090,0.211770,-0.003073,-0.188447,0.117769,0.001613,-0.024223,0.103204,0.032484,0.002688,0.000765,-0.004447,0.148967,-0.018521,-0.014110,-0.001996,-0.002369,-0.120036,0.013172,-0.215571,-0.021999,0.001728,-0.000050,-0.012120,-0.040172,-0.060103,-0.059464,-0.044899,0.015735,0.022919,-0.003106,0.001233,-0.002339,0.040628,0.411684,0.073090,0.526222,0.071060,-0.019531,0.359889,-0.020476,0.057151,0.077110,0.102563,0.188481,-0.016027,-0.135451,-0.189667,0.250967,0.022171,-0.004265
1,-0.714522,-0.042137,-0.052968,-0.796862,-0.018394,0.070102,-0.076321,-0.063864,-1.045521,-0.037353,-0.792515,-1.082483,0.025798,-0.833652,-0.625088,-0.333608,0.072579,-0.046963,0.223022,-0.605902,-0.131099,-0.235929,-0.073920,-0.063247,-0.798768,-0.899983,1.388771,-0.248677,-0.058083,-0.014470,0.092095,0.561368,0.224819,0.223190,0.098852,-0.128227,-0.215876,-0.007164,-0.035260,-0.123911,-0.089751,-0.094963,0.362818,0.011107,-1.506356,-0.573679,-0.955222,-0.818880,-1.063295,-1.022679,-1.336188,-0.612039,-0.061357,-0.482805,-0.017077,1.192135,-0.114981,-0.028074,-0.004451,-0.000536,-0.002288,-0.045597,-0.080639,-0.081924,-0.033862,-0.005111,-0.261836,0.000122,-0.045046,0.999854,-0.008835,-0.122379,-0.199892,0.013615,0.014404,-0.000405,0.021573,-0.024160,-0.037420,-0.012610,0.003007,0.003617,-0.106893,-0.394834,-0.132496,-0.027354,-0.129804,-0.066157,-0.494334,0.123781,0.284328,0.281308,0.212767,0.192042,0.146926,-0.118826,-0.039203,-0.256107,0.176622,0.168840,0.487752,0.029464,0.014232,0.039633,0.025667,0.006626,0.005180,0.006128,-0.016375,0.020727,-0.006525,-0.018790,-0.098543,0.317744,-0.180502,-0.009215
2,0.104791,-0.038188,-0.053191,0.620233,0.148587,0.489875,0.319274,-0.060246,0.053174,-0.025008,-0.456840,1.284450,-0.133470,3.207672,2.373230,1.304427,NaN,NaN,-0.361293,2.995661,NaN,-0.188988,-0.044158,-0.024550,-0.586562,-0.176292,-1.013037,0.066912,0.219649,0.154490,2.370951,1.384675,0.489152,0.484715,0.367301,0.749572,0.669410,0.423228,0.226897,3.227283,-0.329997,-0.327579,-1.033898,0.014531,0.211889,-1.197156,2.860444,NaN,3.584223,NaN,1.272375,7.427558,-0.182816,-2.713205,-1.877595,-0.568691,0.224945,0.052749,0.377640,0.002656,0.001226,0.226060,0.207653,0.270327,0.283061,0.002934,0.454366,0.004264,0.188623,-0.265918,0.000000,2.063796,1.076458,0.240011,NaN,NaN,-0.028327,1.764826,NaN,0.005847,-0.011166,-0.012626,-0.010822,0.056514,-0.100007,-0.216081,-0.127274,-0.056206,0.175751,-0.011770,0.493157,0.487919,0.438576,0.574623,0.564379,-0.165933,-0.051256,0.410379,0.056624,0.047592,0.000000,-0.020586,0.237539,0.017314,0.516667,NaN,0.404158,NaN,0.272937,0.774169,-0.007144,0.123954,0.000000,-0.110103,0.186669,-0.030720
3,-0.532847,-0.006582,-0.023377,1.306702,-0.068909,0.048024,-0.119481,-0.021057,-1.012916,-0.011783,1.206727,0.311773,-0.005928,3.869459,-1.064793,0.107702,-0.126984,-0.044360,-0.181023,-0.691971,NaN,0.195138,-0.104877

## Shape

In [5]:
X_train.shape

(8000, 116)

# Classification Models

In [6]:
voting_estimators = [
    ('RandomForest', RandomForestClassifier(random_state=0)),
    #('DecisionTree', DecisionTreeClassifier(random_state=0)),
    #('SVM', SVC(random_state=0, probability=True)),
    #('NaiveBayes', GaussianNB()),
    #('KNN', KNeighborsClassifier()),
    #('LogisticRegression', LogisticRegression(random_state=0, solver="saga", max_iter=1000)),
    ('AdaBoost', AdaBoostClassifier(random_state=0, algorithm='SAMME')),
    ('GradientBoost', GradientBoostingClassifier(random_state=0)),
    ('XGBoost', XGBClassifier(seed=0)),
    #('NeuralNetwork', MLPClassifier(random_state=0, max_iter=1000)) #hidden_layer_sizes=(20,20) for 2 hidden layers with 20 neurons each
]

vote_model = VotingClassifier(
    estimators=voting_estimators, 
    voting='soft'
)

stacking_estimators = [
    ('RandomForest', RandomForestClassifier(random_state=0)),
    #('DecisionTree', DecisionTreeClassifier(random_state=0)),
    #('SVM', SVC(random_state=0, probability=True)),
    #('NaiveBayes', GaussianNB()),
    #('KNN', KNeighborsClassifier()),
    ('AdaBoost', AdaBoostClassifier(random_state=0, algorithm='SAMME')),
    ('GradientBoost', GradientBoostingClassifier(random_state=0)),
    ('XGBoost', XGBClassifier(seed=0)),
    #('NeuralNetwork', MLPClassifier(random_state=0, max_iter=1000))
]
meta_stack_classifier = LogisticRegression(random_state=0, solver="saga", max_iter=1000)

stacking_model = StackingClassifier(
    estimators=stacking_estimators, 
    final_estimator=meta_stack_classifier, 
    cv=5
)

estimators = [
    ('RandomForest', RandomForestClassifier(random_state=0)),
    #('DecisionTree', DecisionTreeClassifier(random_state=0)),
    #('SVM', SVC(random_state=0, probability=True)),
    #('NaiveBayes', GaussianNB()),
    #('KNN', KNeighborsClassifier()),
    #('LogisticRegression', LogisticRegression(random_state=0, solver="saga", max_iter=1000)),
    ('GradientBoost', GradientBoostingClassifier(random_state=0)),
    ('XGBoost', XGBClassifier(seed=0)),
    ('AdaBoost', AdaBoostClassifier(random_state=0, algorithm='SAMME'))
    #('Voting', vote_model),
    #('Stacking', stacking_model),
    #('NeuralNetwork', MLPClassifier(random_state=0, max_iter=1000)) # 2 hidden layers with 20 neurons each
]

# Create Pipeline with different combination of preprocessing steps

## Combination 6
#### knn imputation, standard scaling, RFECV

In [7]:
objs = dict()

In [8]:
####################################### Imputing Missing Values
X_trainP, imp = handle_missing_vals_knn(
    X_train, 
    n_neighbors=5
)

objs['miss'] = imp

In [9]:
####################################### Standard Scaler
std_scale_cols = (
    X_trainP
    .loc[:, ~X_trainP.columns.str.contains('Group')]
    .columns
)

X_trainP, std_scaler = apply_std_scaler(
    X_trainP, 
    std_scale_cols
)

objs['scaler'] = std_scaler

### Select k best with F score, with 40 features

In [27]:
X_trainP, important_features = select_k_best(X_trainP, y_train, f_classif, 40)

# Apply cross validation with f1, auc and accuracy

#### *Weighted F1 Score: F1 score calculated by taking the average of F1 scores for each class. Average is weighted by support which is the number of true instances for each label. 
#### *AUC One vs One Weighted: By considering all pairwise combinations of classes, average AUC is calculated. Average is weighted by the support. 


## Create Class object and apply cv

In [28]:
error_cost_matrix = np.array([[0, 1, 2],
                              [1, 0, 1],
                              [2, 1, 0]])
model_selector = ModelSelection(
    x_train=X_trainP, 
    y_train=y_train,
    estimators=estimators,
    cost_matrix=error_cost_matrix
)
model_selector.encode_y_train()
model_selector.create_col_transformer()

In [29]:
model_selector.target_label_mapping

{0: -1, 1: 0, 2: 1}

In [30]:
model_selector.calculate_cv_f1(n_folds=5, scoring_average='f1_weighted')
print('**********************************************')
model_selector.calculate_cv_auc(n_folds=5, scoring_average='roc_auc_ovo_weighted')
print('**********************************************')   
model_selector.calculate_cv_accuracy(n_folds=5, scoring_average='accuracy')
print('**********************************************')   
model_selector.calculate_cost_matrix_error_cv(custom_cost_func=matrix_error_function, n_folds=5)

CV Results for Mean F1 Score:

RandomForest = 0.432803
GradientBoost = 0.433581
XGBoost = 0.430447
AdaBoost = 0.420735

Best Estimator (F1): GradientBoost
**********************************************
CV Results for Mean AUC Score:

RandomForest = 0.538182
GradientBoost = 0.543286
XGBoost = 0.532127
AdaBoost = 0.535623

Best Estimator (AUC): GradientBoost
**********************************************
CV Results for Mean Accuracy Score:

RandomForest = 0.468750
GradientBoost = 0.482125
XGBoost = 0.459625
AdaBoost = 0.484000

Best Estimator (Accuracy): AdaBoost
**********************************************
CV Results for Cost Matrix Error Score:

RandomForest = 0.913875
GradientBoost = 0.889750
XGBoost = 0.919875
AdaBoost = 0.890000

Best Estimator (Cost Metric Error Score): GradientBoost


## GridSearchCV

In [37]:
# param_grid = {
#     'ClassificationModel__n_estimators': [100, 200, 250],
#     'ClassificationModel__max_depth': [3, 10, 20],
#     'ClassificationModel__min_samples_split': [2, 10, 20],
#     'ClassificationModel__learning_rate': [0.05, 0.1, 0.2],
#     'ClassificationModel__random_state': [0]
# }

In [38]:
# model_selector.apply_grid_cv(
#     estimator=GradientBoostingClassifier(),
#     params=param_grid,
#     cv=2,
#     scoring=matrix_error_function
# )

### Predict on real dataset

In [39]:
# ######################################## Imputing Missing Values
# X_test_compete = pd.DataFrame(objs['miss'].transform(X_test_compete), columns=X_test_compete.columns)
# ######################################## Scaling
# X_test_compete = pd.DataFrame(objs['scaler'].transform(X_test_compete[num_cols_test]), columns=num_cols_test).reset_index(drop=True)
# ######################################## RFECV
# subset_features_rfecv = X_trainP.columns.to_list()
# X_test_compete = X_test_compete.loc[:, subset_features_rfecv]

In [43]:
preds = pd.DataFrame(grid_search2.grid_best_model.predict(X_test_compete)).replace(grid_search2.target_label_mapping)

In [44]:
file_path = "preds_comb4_5.txt"
preds.to_csv(file_path, index=False, header=False)

### Select K-best with mutual_info_classif, with 40 features

In [34]:
objs = dict()

####################################### Imputing Missing Values
X_trainP, imp = handle_missing_vals_knn(
    X_train, 
    n_neighbors=5
)

objs['miss'] = imp

####################################### Standard Scaler
std_scale_cols = (
    X_trainP
    .loc[:, ~X_trainP.columns.str.contains('Group')]
    .columns
)

X_trainP, std_scaler = apply_std_scaler(
    X_trainP, 
    std_scale_cols
)

objs['scaler'] = std_scaler

X_trainP, important_features = select_k_best(X_trainP, y_train, mutual_info_classif, 40)

In [35]:
model_selector2 = ModelSelection(
    x_train=X_trainP, 
    y_train=y_train,
    estimators=estimators,
    cost_matrix=error_cost_matrix
)
model_selector2.encode_y_train()
model_selector2.create_col_transformer()

In [36]:
model_selector2.calculate_cv_f1(n_folds=5, scoring_average='f1_weighted')
print('**********************************************')
model_selector2.calculate_cv_auc(n_folds=5, scoring_average='roc_auc_ovo_weighted')
print('**********************************************')   
model_selector2.calculate_cv_accuracy(n_folds=5, scoring_average='accuracy')
print('**********************************************')   
model_selector2.calculate_cost_matrix_error_cv(custom_cost_func=matrix_error_function, n_folds=5)

CV Results for Mean F1 Score:

RandomForest = 0.432847
GradientBoost = 0.426424
XGBoost = 0.438818
AdaBoost = 0.409358

Best Estimator (F1): XGBoost
**********************************************
CV Results for Mean AUC Score:

RandomForest = 0.541991
GradientBoost = 0.547004
XGBoost = 0.539158
AdaBoost = 0.530348

Best Estimator (AUC): GradientBoost
**********************************************
CV Results for Mean Accuracy Score:

RandomForest = 0.470875
GradientBoost = 0.474875
XGBoost = 0.467500
AdaBoost = 0.474125

Best Estimator (Accuracy): GradientBoost
**********************************************
CV Results for Cost Matrix Error Score:

RandomForest = 0.911250
GradientBoost = 0.903375
XGBoost = 0.908125
AdaBoost = 0.909750

Best Estimator (Cost Metric Error Score): GradientBoost


### Select k best with F score, best 80 features

In [42]:
objs = dict()

####################################### Imputing Missing Values
X_trainP, imp = handle_missing_vals_knn(
    X_train, 
    n_neighbors=5
)

objs['miss'] = imp

####################################### Standard Scaler
std_scale_cols = (
    X_trainP
    .loc[:, ~X_trainP.columns.str.contains('Group')]
    .columns
)

X_trainP, std_scaler = apply_std_scaler(
    X_trainP, 
    std_scale_cols
)

objs['scaler'] = std_scaler

X_trainP, important_features = select_k_best(X_trainP, y_train, mutual_info_classif, 40)

In [43]:
model_selector3 = ModelSelection(
    x_train=X_trainP, 
    y_train=y_train,
    estimators=estimators,
    cost_matrix=error_cost_matrix
)
model_selector3.encode_y_train()
model_selector3.create_col_transformer()

In [44]:
model_selector3.calculate_cv_f1(n_folds=5, scoring_average='f1_weighted')
print('**********************************************')
model_selector3.calculate_cv_auc(n_folds=5, scoring_average='roc_auc_ovo_weighted')
print('**********************************************')   
model_selector3.calculate_cv_accuracy(n_folds=5, scoring_average='accuracy')
print('**********************************************')   
model_selector3.calculate_cost_matrix_error_cv(custom_cost_func=matrix_error_function, n_folds=5)

CV Results for Mean F1 Score:

RandomForest = 0.437851
GradientBoost = 0.435154
XGBoost = 0.438532
AdaBoost = 0.411039

Best Estimator (F1): XGBoost
**********************************************
CV Results for Mean AUC Score:

RandomForest = 0.543651
GradientBoost = 0.550516
XGBoost = 0.535113
AdaBoost = 0.538311

Best Estimator (AUC): GradientBoost
**********************************************
CV Results for Mean Accuracy Score:

RandomForest = 0.476250
GradientBoost = 0.482000
XGBoost = 0.469000
AdaBoost = 0.477750

Best Estimator (Accuracy): GradientBoost
**********************************************
CV Results for Cost Matrix Error Score:

RandomForest = 0.901500
GradientBoost = 0.889500
XGBoost = 0.904375
AdaBoost = 0.902500

Best Estimator (Cost Metric Error Score): GradientBoost
